In [ ]:
from owslib.wfs import WebFeatureService

In [ ]:
wfs11 = WebFeatureService(url='https://www.imis.bfs.de/ogc/opendata/ows', version='1.1.0')
wfs11.identification.title

In [ ]:
[operation.name for operation in wfs11.operations]

In [ ]:
list(wfs11.contents)

In [ ]:
wfs11.get_schema('odlinfo_timeseries_precipitation_15min')

In [ ]:
import xml.dom.minidom # WFS returns GML - this library is used to parse it

response = wfs11.getfeature(typename='opendata:odlinfo_timeseries_precipitation_15min', bbox=(7.089845,45.253298,12.166505,55.075436), srsname='EPSG:4326')
xml = xml.dom.minidom.parse(response) # or xml.dom.minidom.parseString(xml_string)
print(xml.toprettyxml(indent="    "))

In [ ]:
import xml.dom.minidom # WFS returns GML - this library is used to parse it

response = wfs11.getfeature(typename='opendata:odlinfo_timeseries_odl_24h', bbox=(7.089845,45.253298,12.166505,55.075436), srsname='EPSG:4326',maxfeatures=5)
xml = xml.dom.minidom.parse(response) # or xml.dom.minidom.parseString(xml_string)
print(xml.toprettyxml(indent="    "))

In [ ]:
from owslib.fes import *
from owslib.etree import etree
import xml.dom.minidom # WFS returns GML - this library is used to parse it

# apply filter by property name and value
filter = property
filter=PropertyIsGreaterThanOrEqualTo(propertyname='opendata:end_measure', literal='2021-07-29T00:00:000Z')
filterxml = etree.tostring(filter.toXML()).decode("utf-8")

response = wfs11.getfeature(typename='odlinfo_timeseries_odl_24h',filter = filterxml, maxfeatures=5)

xml = xml.dom.minidom.parse(response) # or xml.dom.minidom.parseString(xml_string)
print(xml.toprettyxml(indent="    "))

In [ ]:
from owslib.fes import *
from owslib.etree import etree
import xml.dom.minidom # WFS returns GML - this library is used to parse it

# apply filter by property name and value
filter = PropertyIsLike(propertyname='opendata:nuclide', literal='Cs-137', wildCard='*')
filterxml = etree.tostring(filter.toXML()).decode("utf-8")

response = wfs11.getfeature(typename='opendata:nuklide_fleisch',filter = filterxml, maxfeatures=5)

xml = xml.dom.minidom.parse(response) # or xml.dom.minidom.parseString(xml_string)
print(xml.toprettyxml(indent="    "))

In [ ]:
import json

response = wfs11.getfeature(typename='opendata:nuklide_fleisch',bbox=(7.089845,45.253298,12.166505,55.075436), srsname='EPSG:4326', outputFormat='application%2Fjson',maxfeatures=200)
# convert IO-byte to bytes
bytesD=bytes(response.read())
# convert to json
data = json.loads(bytesD)
print(data)

In [ ]:
from owslib.fes import *
from owslib.etree import etree
import json

#apply filter by property name and value
filter = PropertyIsLike(propertyname='name', literal='Flensburg', wildCard='*')
filterxml = etree.tostring(filter.toXML()).decode("utf-8")
#get feature
response = wfs11.getfeature(typename='opendata:odlinfo_timeseries_odl_1h', filter=filterxml,bbox=(7.089845,45.253298,12.166505,55.075436), srsname='EPSG:4326', outputFormat='application%2Fjson',maxfeatures=5)

bytesD=bytes(response.read())# convert IO-byte to bytes
data = json.loads(bytesD)# load json
print(data)

In [ ]:
import json # WFS returns GML - this library is used to parse it
from owslib.fes import *
from owslib.etree import etree

#apply multifilter
filterxml = "<ogc:Filter><ogc:PropertyIsEqualTo><ogc:PropertyName>name</ogc:PropertyName><ogc:Literal>Flensburg</ogc:Literal></ogc:PropertyIsEqualTo><ogc:PropertyIsEqualTo><ogc:PropertyName>Kenn</ogc:PropertyName><ogc:Literal>010010001</ogc:Literal></ogc:PropertyIsEqualTo></ogc:Filter>"   # create the filter for the species
#filter = PropertyIsGreaterThan(propertyname='value', literal='3')
#filter = PropertyIsLessThan(propertyname='end_measure', literal='2022-07-28T15:00:00Z') and PropertyIsLike(propertyname='locality_name', literal='Lögow', wildCard='*')
#filter = PropertyIsLessThanOrEqualTo(propertyname='opendata:end_measure', literal='2022-07-23T19:00:000Z')
#filter = PropertyIsEqualTo(propertyname='kenn', literal='083370490')
#filter = PropertyIsLike(propertyname='opendata:name', literal='Krün OT Elmau', wildCard='*')
#filter = PropertyIsLike(propertyname='bez_gem', literal='Ingolstadt', wildCard='*')
#WebFeatureService_1_1_0.getfeature(self, typename, filter, bbox, featureid, featureversion, propertyname, maxfeatures, srsname, outputFormat, method, startindex, sortby)

#filterxml = etree.tostring(filter.toXML()).decode("utf-8")

response = wfs11.getfeature(typename='opendata:odlinfo_timeseries_precipitation_15min', filter=filterxml, maxfeatures=5, outputFormat='application/json', method='get')
# convert IO-byte to bytes
bytesD=bytes(response.read())
# Load json
data = json.loads(bytesD)
print(data)

In [ ]:
#save geojson in qgis
import geojson

outFileName='geotest.geojson'
geom_in_geojson = geojson.Feature(geometry=data, properties={})
outFile=open(outFileName, "w")
with open(outFileName, 'w') as outfile:
      geojson.dump(geom_in_geojson, outfile)
outFile.close()


In [ ]:
#save geojson and add layer in qgis
import os # This is is needed in the pyqgis console also
from qgis.core import (
    QgsVectorLayer
)
from qgis.core import QgsProject

# get the path to the shapefile e.g. /home/project/data/ports.shp
path_to_layer = "geotest.geojson"

# The format is:
# vlayer = QgsVectorLayer(data_source, layer_name, provider_name)

vlayer = QgsVectorLayer(path_to_layer, "bfsgeo", "ogr")
if not vlayer.isValid():
    print("Layer failed to load!")
else:
    QgsProject.instance().addMapLayer(vlayer)

In [ ]:
#save json to show in qgis
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:

import pandas as pd
from matplotlib import pyplot as plt
import requests
import json



#df = pd.DataFrame(data=data)
#print(df)

# using pandas for serializing
df= pd.json_normalize(data,"features")
print(df)

dfg = df.groupby('properties.id').count()
print(dfg)
# using matplotlib scatter chart
plt.bar(dfg.index, dfg['properties.value'], width=0.5, color="maroon")
plt.show()

# using pandas plot
dfg.plot()